In [24]:
import pandas as pd
import os

In [25]:
base = 'zibs2020'
folder = os.path.join(base, 'downloads')
files = os.listdir(folder)
files

['Nl.zorg.Alert-v4.1(2020EN).xlsx',
 'Nl.zorg.AllergyIntolerance-v3.3(2020EN).xlsx',
 'Nl.zorg.HealthcareProvider-v3.4(2020EN).xlsx',
 'Nl.zorg.HealthProfessional-v3.5(2020EN).xlsx',
 'Nl.zorg.MedicalDevice-v3.3.1(2020EN).xlsx',
 'Nl.zorg.Patient-v3.2(2020EN).xlsx',
 'Nl.zorg.Problem-v4.4(2020EN).xlsx',
 'Nl.zorg.Procedure-v5.2(2020EN).xlsx']

In [26]:
file = 'Nl.zorg.Patient-v3.2(2020EN).xlsx'
url = os.path.join(folder, file)
zibdf = pd.read_excel(url, sheet_name='Data', header=1)
zibdf

c:\Users\marcd\Anaconda3\envs\fhir\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Concept,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Alias,Type,Card.,Stereotype,Id,Definition,DefinitionCode,Reference,Constraints
0,NaN,Patient,NaN,NaN,NaN,NaN,NaN,NL: Patient,NaN,NaN,rootconcept,NL-CM:0.1.1,Root concept of the Patient information model....,SNOMED CT: 116154003 Patient,NaN,NaN
1,NaN,NaN,NameInformation,NaN,NaN,NaN,NaN,NL: Naamgegevens,NaN,0..1,"data,reference",NL-CM:0.1.6,Patient's full name.,NaN,This is a reference to the rootconcept of part...,NaN
2,NaN,NaN,AddressInformation,NaN,NaN,NaN,NaN,NL: Adresgegevens,NaN,0..*,"data,reference",NL-CM:0.1.4,Patient's address information.,NaN,This is a reference to the rootconcept of the ...,NaN
3,NaN,NaN,ContactInformation,NaN,NaN,NaN,NaN,NL: Contactgegevens,NaN,0..1,"data,reference",NL-CM:0.1.5,Patient’s telephone number(s) or e-mail addres...,NaN,This is a reference to the rootconcept of part...,NaN
4,NaN,NaN,PatientIdentificationNumber,NaN,NaN,NaN,NaN,NL: Identificatienummer,II,0..*,data,NL-CM:0.1.7,The patient’s identification number. In transf...,NaN,Burgerservicenummer,NaN
5,NaN,NaN,DateOfBirth,NaN,NaN,NaN,NaN,NL: Geboortedatum,TS,0..1,data,NL-CM:0.1.10,Patient’s date of birth. The date of birth is ...,LOINC: 21112-8 Birth date,NaN,NaN
6,NaN,NaN,Gender,NaN,NaN,NaN,NaN,NL: Geslacht,CD,0..1,data,NL-CM:0.1.9,Patient’s administrative gender.,LOINC: 46098-0 Sex,GenderCodelist,NaN
7,NaN,NaN,MultipleBirthIndicator,NaN,NaN,NaN,NaN,NL: MeerlingIndicator,BL,0..1,data,NL-CM:0.1.31,An indication stating whether the patient is o...,NaN,NaN,NaN
8,NaN,NaN,DeathIndicator,NaN,NaN,NaN,NaN,NL: OverlijdensIndicator,BL,0..1,data,NL-CM:0.1.32,An indication stating whether the patient has ...,SNOMED CT: 397709008 Patient died,NaN,NaN
9,NaN,NaN,DateOfDeath,NaN,NaN,NaN,NaN,NL: DatumOverlijden,TS,0..1,data,NL-CM:0.1.33,The date on which the patient died. A ‘vague’ ...,LOINC: 81954-0 Date of death [Date],NaN,NaN


In [27]:
zibdf.columns = [col.lower() for col in zibdf.columns]
zibdf = zibdf.drop(labels=['unnamed: 0', 'reference'], axis=1).dropna(how='all')
zibdf.insert(0, 'name', None)
zibdf.insert(0, 'path', None)
zibdf

,path,name,concept,unnamed: 2,unnamed: 3,unnamed: 4,unnamed: 5,unnamed: 6,alias,type,card.,stereotype,id,definition,definitioncode,constraints
0,None,None,Patient,NaN,NaN,NaN,NaN,NaN,NL: Patient,NaN,NaN,rootconcept,NL-CM:0.1.1,Root concept of the Patient information model....,SNOMED CT: 116154003 Patient,NaN
1,None,None,NaN,NameInformation,NaN,NaN,NaN,NaN,NL: Naamgegevens,NaN,0..1,"data,reference",NL-CM:0.1.6,Patient's full name.,NaN,NaN
2,None,None,NaN,AddressInformation,NaN,NaN,NaN,NaN,NL: Adresgegevens,NaN,0..*,"data,reference",NL-CM:0.1.4,Patient's address information.,NaN,NaN
3,None,None,NaN,ContactInformation,NaN,NaN,NaN,NaN,NL: Contactgegevens,NaN,0..1,"data,reference",NL-CM:0.1.5,Patient’s telephone number(s) or e-mail addres...,NaN,NaN
4,None,None,NaN,PatientIdentificationNumber,NaN,NaN,NaN,NaN,NL: Identificatienummer,II,0..*,data,NL-CM:0.1.7,The patient’s identification number. In transf...,NaN,NaN
5,None,None,NaN,DateOfBirth,NaN,NaN,NaN,NaN,NL: Geboortedatum,TS,0..1,data,NL-CM:0.1.10,Patient’s date of birth. The date of birth is ...,LOINC: 21112-8 Birth date,NaN
6,None,None,NaN,Gender,NaN,NaN,NaN,NaN,NL: Geslacht,CD,0..1,data,NL-CM:0.1.9,Patient’s administrative gender.,LOINC: 46098-0 Sex,NaN
7,None,None,NaN,MultipleBirthIndicator,NaN,NaN,NaN,NaN,NL: MeerlingIndicator,BL,0..1,data,NL-CM:0.1.31,An indication stating whether the patient is o...,NaN,NaN
8,None,None,NaN,DeathIndicator,NaN,NaN,NaN,NaN,NL: OverlijdensIndicator,BL,0..1,data,NL-CM:0.1.32,An indication stating whether the patient has ...,SNOMED CT: 397709008 Patient died,NaN
9,None,None,NaN,DateOfDeath,NaN,NaN,NaN,NaN,NL: DatumOverlijden,TS,0..1,data,NL-CM:0.1.33,The date on which the patient died. A ‘vague’ ...,LOINC: 81954-0 Date of death [Date],NaN


In [28]:
concepts = []
for i, row in zibdf.iterrows():
    # Loop thru columns 2 tot 8 (Concept and 'Unnamed' ones)
    for j in range(2, 8):
        if not pd.isna(row.iloc[j]):
            # print(j, row.iloc[j])
            try:
                concepts[j - 2] = row.iloc[j]
            except IndexError:
                concepts.append(row.iloc[j])
    # print(i, concepts)
    zibdf.at[i, 'path'] = '.'.join(concepts)
    zibdf.at[i, 'name'] = concepts[-1]

In [29]:
zibdf = zibdf[['name', 'path', 'alias', 'type', 'card.', 'stereotype', 'id', 'definition', 'definitioncode']].fillna('')
zibdf

,name,path,alias,type,card.,stereotype,id,definition,definitioncode
0,Patient,Patient,NL: Patient,,,rootconcept,NL-CM:0.1.1,Root concept of the Patient information model....,SNOMED CT: 116154003 Patient
1,NameInformation,Patient.NameInformation,NL: Naamgegevens,,0..1,"data,reference",NL-CM:0.1.6,Patient's full name.,
2,AddressInformation,Patient.AddressInformation,NL: Adresgegevens,,0..*,"data,reference",NL-CM:0.1.4,Patient's address information.,
3,ContactInformation,Patient.ContactInformation,NL: Contactgegevens,,0..1,"data,reference",NL-CM:0.1.5,Patient’s telephone number(s) or e-mail addres...,
4,PatientIdentificationNumber,Patient.PatientIdentificationNumber,NL: Identificatienummer,II,0..*,data,NL-CM:0.1.7,The patient’s identification number. In transf...,
5,DateOfBirth,Patient.DateOfBirth,NL: Geboortedatum,TS,0..1,data,NL-CM:0.1.10,Patient’s date of birth. The date of birth is ...,LOINC: 21112-8 Birth date
6,Gender,Patient.Gender,NL: Geslacht,CD,0..1,data,NL-CM:0.1.9,Patient’s administrative gender.,LOINC: 46098-0 Sex
7,MultipleBirthIndicator,Patient.MultipleBirthIndicator,NL: MeerlingIndicator,BL,0..1,data,NL-CM:0.1.31,An indication stating whether the patient is o...,
8,DeathIndicator,Patient.DeathIndicator,NL: OverlijdensIndicator,BL,0..1,data,NL-CM:0.1.32,An indication stating whether the patient has ...,SNOMED CT: 397709008 Patient died
9,DateOfDeath,Patient.DateOfDeath,NL: DatumOverlijden,TS,0..1,data,NL-CM:0.1.33,The date on which the patient died. A ‘vague’ ...,LOINC: 81954-0 Date of death [Date]


In [30]:
url = os.path.join(base, 'mappable', file)
zibdf.to_excel(url, index=False)